In [1]:
import torch
from sklearn.datasets import fetch_california_housing
from src.mssp import MSSP

In [2]:
data = fetch_california_housing()
X = torch.tensor(data.data, dtype=torch.double)
y = torch.tensor(data.target, dtype=torch.double)

In [3]:
X.shape, y.shape

(torch.Size([20640, 8]), torch.Size([20640]))

In [4]:
model = MSSP(n_best=120, loss_fn="mse", random_seed=42, epochs=3, diversity_ratio=0.75, pow_cross=True)

In [5]:
model.fit(X, y)

loss (mse): 0.6123 epoch: 0 , time: 1.10s
loss (mse): 0.4738 epoch: 1 , time: 2.83s
loss (mse): 0.4640 epoch: 2 , time: 2.58s
Best loss: 0.4639781415462494 after training for 2 epochs


In [6]:
model._build_model(3)

In [10]:
y_pred = model.predict(X)

In [15]:
((y_pred - y)**2)

KeyboardInterrupt: 

In [ ]:
y_pred